In [ ]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

### Setup and Configuration

In [ ]:
# Parameters for 3-Hour Prediction
windowsize = 24      # Use the past 24 hours of data
prediction_step = 3  # Predict the return 3 hours into the future

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Set seeds for reproducibility
np.random.seed(1234)
torch.manual_seed(1234)
random.seed(1234)

### Data Loading and Preprocessing

In [ ]:
print("Loading and resampling data...")
df_full = pd.read_csv('../data/btcusd_1-min_data.csv')

# Convert timestamp to datetime
df_full['timestamp'] = pd.to_datetime(df_full['Timestamp'], unit='s')
df_full = df_full.set_index('timestamp')

# Resample to hourly frequency instead of daily
# approximate hourly close with last()
df_hourly = df_full['Close'].resample('h').last().to_frame()

# Drop hours with no data
df_hourly = df_hourly.dropna()

print(f"Resampled to {len(df_hourly)} hourly data points.")

### Features

In [ ]:
# Calculate hourly log returns
df_hourly['log_return'] = np.log(df_hourly['Close'] / df_hourly['Close'].shift(1))
df_hourly = df_hourly.dropna().reset_index()

print(f"Calculated hourly log returns. Shape: {df_hourly.shape}")

### Data Preparation

In [ ]:
# -- Data Splitting (using hourly data) --
split_fraction = 0.8
split_idx = int(len(df_hourly) * split_fraction)
train_data_raw = df_hourly.iloc[:split_idx].copy()
test_data_raw = df_hourly.iloc[split_idx:].copy().reset_index(drop=True)

print(f"Raw hourly training data shape: {train_data_raw.shape}")
print(f"Raw hourly test data shape: {test_data_raw.shape}")

# -- Normalization (using hourly log returns) --
scaler = StandardScaler()
# Fit scaler ONLY on training data's hourly log_return
scaler.fit(train_data_raw['log_return'].values.reshape(-1, 1))

# Apply scaler to both train and test data
train_data_raw['scaled_log_return'] = scaler.transform(train_data_raw['log_return'].values.reshape(-1, 1))
test_data_raw['scaled_log_return'] = scaler.transform(test_data_raw['log_return'].values.reshape(-1, 1))

# -- Create Lagged Data Function (Now works for hourly steps) --
def create_lagged_data(data, windowsize, prediction_step, feature_col='scaled_log_return', target_col='log_return', device=None):
    """
    Create lagged data for predicting a specific future step (now hourly).
    Args:
        data: DataFrame with feature_col and target_col (hourly data)
        windowsize: Number of past hours for input features
        prediction_step: How many hours ahead to predict (3 for 3-hour prediction)
        feature_col: Column name for input features (e.g., 'scaled_log_return')
        target_col: Column name for target variable (e.g., 'log_return')
        device: Device to place tensors on
    Returns:
        x, y: PyTorch tensors of inputs (scaled) and targets (unscaled)
    """
    x, y = [], []
    # Loop stops early enough for windowsize and prediction_step
    for i in range(len(data) - windowsize - prediction_step + 1):
        # Input features: PAST 'windowsize' scaled hourly returns
        feature = data[feature_col].iloc[i : i + windowsize].values

        # Target: SUM OF ACTUAL (unscaled) returns for the next 3 hours
        # We're predicting the CUMULATIVE return over the next 3 hours
        target_indices = range(i + windowsize, i + windowsize + prediction_step)
        target = data[target_col].iloc[target_indices].sum()  # Sum of next 3 hourly returns

        x.append(feature)
        y.append(target)

    x = np.array(x)
    y = np.array(y)

    x_tensor = torch.FloatTensor(x).to(device)
    y_tensor = torch.FloatTensor(y).to(device)

    return x_tensor, y_tensor

### Training and Testing Datasets

In [ ]:
# -- Create Datasets (using hourly data) --
x_train, y_train = create_lagged_data(
    train_data_raw, windowsize, prediction_step,
    feature_col='scaled_log_return', target_col='log_return', device=device
)
x_test, y_test = create_lagged_data(
    test_data_raw, windowsize, prediction_step,
    feature_col='scaled_log_return', target_col='log_return', device=device
)

print(f"Training input shape: {x_train.shape}, target shape: {y_train.shape}")
print(f"Test input shape: {x_test.shape}, target shape: {y_test.shape}")

# Check if datasets are empty (can happen if not enough hourly data)
if x_train.shape[0] == 0 or x_test.shape[0] == 0:
    raise ValueError("Created datasets are empty. Check data length after resampling and window/prediction steps.")

### Data Loader and Model Definition

In [ ]:
# -- DataLoader --
batch_size = 64  # Increased since hourly data will give us more samples
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

# -- Model Definition --
class RNN_model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.rnn = torch.nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.1
        )
        self.fc1 = torch.nn.Linear(hidden_size, 30)
        self.fc2 = torch.nn.Linear(30, 1)

    def forward(self, x):
        x = x.unsqueeze(-1) # Shape: (batch, seq_len=windowsize, input_size=1)
        out, _ = self.rnn(x)
        out = out[:, -1, :] # Shape: (batch, hidden_size)
        out = torch.nn.functional.relu(self.fc1(out))
        out = self.fc2(out)
        return out.squeeze(-1) # Shape: (batch)


# -- Model, Optimizer, Loss (Adjusted Hyperparameters) --
input_size = 1
hidden_size = 32
num_layers = 2
learning_rate = 0.001
n_epochs = 30

model = RNN_model(input_size, hidden_size, num_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()

### Training Definition

In [ ]:
# -- Training Loop Function --
def train_RNN(model, n_epochs, loader, optimizer, loss_fn, x_train, x_test, y_train, y_test, device):
    train_losses = []
    test_rmses = []
    eval_batch_size = 128

    print("Starting Training...")
    for epoch in range(n_epochs):
        model.train()
        batch_losses = []
        for x_batch, y_batch in loader:
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())

        epoch_loss = np.mean(batch_losses)
        train_losses.append(epoch_loss)

        # Validation
        if epoch % 5 == 0 or epoch == n_epochs - 1:
            model.eval()
            with torch.no_grad():
                # Evaluate on Training Set
                train_preds_list = []
                # Handle potential smaller train set in eval loop
                current_pos = 0
                while current_pos < len(x_train):
                    x_batch_eval = x_train[current_pos:min(current_pos + eval_batch_size, len(x_train))]
                    if x_batch_eval.shape[0] > 0: # Ensure batch not empty
                         batch_preds = model(x_batch_eval)
                         train_preds_list.append(batch_preds)
                    current_pos += eval_batch_size
                if not train_preds_list: # Handle case where train set is smaller than eval_batch_size
                     train_rmse = float('nan')
                else:
                     y_pred_train = torch.cat(train_preds_list)
                     train_rmse = torch.sqrt(loss_fn(y_pred_train, y_train))

                # Evaluate on Test Set
                test_preds_list = []
                current_pos = 0
                while current_pos < len(x_test):
                     x_batch_eval = x_test[current_pos:min(current_pos + eval_batch_size, len(x_test))]
                     if x_batch_eval.shape[0] > 0: # Ensure batch not empty
                         batch_preds = model(x_batch_eval)
                         test_preds_list.append(batch_preds)
                     current_pos += eval_batch_size
                if not test_preds_list: # Handle case where test set is smaller than eval_batch_size
                    test_rmse = float('nan')
                    test_rmses.append(test_rmse)
                else:
                    y_pred_test = torch.cat(test_preds_list)
                    test_rmse = torch.sqrt(loss_fn(y_pred_test, y_test))
                    test_rmses.append(test_rmse.item())


            print(f"Epoch {epoch}: Train Loss {epoch_loss:.6f}, Train RMSE {train_rmse:.6f}, Test RMSE {test_rmse:.6f}")
    print("Training Finished.")
    return train_losses, test_rmses

### Training

In [ ]:
# -- Train the Model --
train_losses, test_rmses = train_RNN(
    model, n_epochs, train_loader, optimizer, loss_fn,
    x_train, x_test, y_train, y_test, device
)

### Evaluation

In [ ]:
# -- Evaluation --
model.eval()
with torch.no_grad():
    y_pred_test_final = model(x_test).cpu().numpy()

y_actual_test = y_test.cpu().numpy()

# Calculate final metrics
test_mse = mean_squared_error(y_actual_test, y_pred_test_final)
test_r2 = r2_score(y_actual_test, y_pred_test_final)
print(f"\nFinal Test MSE: {test_mse:.8f}")
print(f"Final Test R²: {test_r2:.6f}")

### Strategy

In [ ]:
# -- Simple Trading Strategy (3-Hour) --
def generate_trading_signals(predictions, threshold=0):
    signals = np.zeros_like(predictions)
    signals[predictions > threshold] = 1
    signals[predictions < -threshold] = -1
    return signals

# Generate signals
signals = generate_trading_signals(y_pred_test_final, threshold=0) # Trade on any predicted direction
print(f"Trading signals generated. Shape: {signals.shape}")
print(f"Signal distribution: Long: {np.sum(signals > 0)}, Short: {np.sum(signals < 0)}, Hold: {np.sum(signals == 0)}")

# Calculate strategy returns (signal * actual next 3-hour return)
strategy_returns = signals * y_actual_test
print(f"Strategy returns calculated. Shape: {strategy_returns.shape}")

# Calculate cumulative returns
cumulative_strategy_returns = np.cumsum(strategy_returns)
cumulative_benchmark_returns = np.cumsum(y_actual_test)

# Calculate Sharpe ratio (annualized for hourly data)
if np.std(strategy_returns) > 1e-9:
    # Annualization factor for hourly trading (approx 24*252 = 6048 trading hours per year)
    trading_hours_per_year = 24 * 252
    sharpe_ratio = (np.mean(strategy_returns) / np.std(strategy_returns)) * np.sqrt(trading_hours_per_year)
else:
    sharpe_ratio = 0.0
    print("Warning: Standard deviation of strategy returns is zero or near-zero.")

print(f"Strategy Sharpe Ratio (Annualized): {sharpe_ratio:.4f}")

In [ ]:
# -- Plotting (3-Hour) --
plt.figure(figsize=(14, 7))
# Use test data timestamps for x-axis if available and aligned
test_dates = test_data_raw.iloc[windowsize + prediction_step - 1 : windowsize + prediction_step -1 + len(cumulative_strategy_returns)]['timestamp']

if len(test_dates) == len(cumulative_strategy_returns):
    plt.plot(test_dates, cumulative_strategy_returns, label=f'RNN Strategy (Predict 3 hours ahead)', color='cyan')
    plt.plot(test_dates, cumulative_benchmark_returns, label='Buy & Hold (Benchmark)', color='orange')
    plt.xlabel('Date')
else:
    # Fallback to plotting against index if dates don't align perfectly
    print("Warning: Test dates length mismatch, plotting against index.")
    plt.plot(cumulative_strategy_returns, label=f'RNN Strategy (Predict 3 hours ahead)', color='cyan')
    plt.plot(cumulative_benchmark_returns, label='Buy & Hold (Benchmark)', color='orange')
    plt.xlabel('Trading Hours (Test Set)')


plt.legend()
plt.title(f'Cumulative Returns (3-Hour): RNN Strategy vs Buy & Hold (Window={windowsize} hours)')
plt.ylabel('Cumulative Log Returns')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs (Hourly Data)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()